In [1]:
# Import libraries

import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import re
import pandas as pd
from sqlalchemy import create_engine
import lxml
from pprint import pprint
from datetime import datetime

import os
import time

# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options

In [ ]:
url = 'https://www.trustpilot.com/categories/restaurants_bars?sort=latest_review'
       #https://www.trustpilot.com/categories/restaurants_bars?page=2&sort=latest_review
       # an average of 6 pages covers a 2 weeks scraping interval

response = requests.get(url)

print(response)

<Response [200]>


In [3]:
print(response.text)

<!DOCTYPE html><html lang="en-US"><head><meta charSet="UTF-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="shortcut icon" type="image/x-icon" href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon.ico"/><link rel="manifest" href="/manifest.json"/><meta name="application-name" content="Trustpilot"/><meta name="theme-color" content="#1c1c1c"/><link rel="apple-touch-icon" sizes="180x180" href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/apple-touch-icon.png"/><link rel="icon" type="image/png" sizes="32x32" href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon-32x32.png"/><link rel="icon" type="image/png" sizes="16x16" href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon-16x16.png"/><link rel="mask-icon" href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/safari-pinned-tab.svg" color="#00b67a"/><meta name="apple-mobile-web-app-title" content="Trustpilot"/><meta name="msapplication-TileColor

In [4]:
# Request with BeatutifulSoup

soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="Trustpilot" name="application-name"/>
  <meta content="#1c1c1c" name="theme-color"/>
  <link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link color="#00b67a" href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="Trustpilot" name="apple-mobile-web-app-title"/

In [5]:
data = soup.find_all('a', href=re.compile(r'^/review/'))

# Create an empty set to hold unique hrefs
recent_companies_reviews = set()

# Use a normal for loop to add each href to the set.
for link in data:
    recent_companies_reviews.add(link.get('href'))


for recent in recent_companies_reviews:
    print(recent)

print("\n")

print(len(recent_companies_reviews))



/review/www.grubhub.com
/review/www.restockit.com
/review/everyplate.com
/review/smokehouse.com
/review/exoticsnacks.com
/review/www.ezcater.com
/review/factor75.com
/review/www.slimchickens.co.uk
/review/modifyhealth.com
/review/mc-donalds.com
/review/www.starbucks.com
/review/www.kfc.com
/review/plantshed.com
/review/laboutiquedeschefs.com
/review/www.fazolis.com
/review/rosegarnet.com
/review/www.magickitchen.com
/review/hellofresh.com
/review/texaslonestartamales.com
/review/fultonfishmarket.com


20


In [152]:
# test_review = list(recent_companies_reviews)
# # test_review = [test_review[0], test_review[1]]
# test_review = [test_review[0], test_review[1]]

# # print(test_review)




# Creating an empty dictionary to store the parameters to be extracted
reviews_dict = {
    'company_name' : [],
    'country' : [],
    'reviewer_name' : [],
    'review_title' : [],
    'review_text' : [],
    'Date_of_experience' : [],
    'rating_text' : []
}


# Looping through the list of companies with recent reviews
for recent in recent_companies_reviews:
    new_url = 'https://www.trustpilot.com' + recent

    Response = requests.get(new_url)

    print(Response)

    print("\n")

    # Request with BeatutifulSoup
    Soup = BeautifulSoup(Response.text, 'html.parser')
    # print(Soup.prettify())
    # print("\n")


    # Now we're on each of the companies's page, we'll then extract all the 7 details from the page starting with...
    # 1, The company's name
    company_name = Soup.find('h1', class_ = 'typography_default__PeVD_ typography_appearance-default__t8iAq title_title__dK8Bt').text
    company_name = company_name.split()
    company_name1 = company_name[0]
    print(f'The company_name is: {company_name[0]}')
    print("\n")

    
    # 2, Find the relevant div that contains all the reviews on the company's review page
    reviews = Soup.find_all('div', class_ = 'styles_cardWrapper__kOLEb styles_show__qAseP')

    for review in reviews:
        
        if review:
            try:
                # 2, Find the relevant div that contains the reviewer's country
                country = review.find('div', class_ = 'styles_consumerExtraDetails__TylYM').text
                country = country.split('•')
                country1 = country[0]
                print(f'The country is: {country[0]}')
                print("\n")

            except AttributeError as e:
                print(f"Error parsing Country from {review}: {e}")
                continue  # Skip this job if data parsing fails

        if review:
            try:
                # 3, Find the relevant div that contains the reviewer's name
                reviewer_name = review.find('a', href=re.compile(r'^/users/')).text
                reviewer_name = reviewer_name.split(country1)
                reviewer_name1 = reviewer_name[0]
                print(f'The reviewer_name is: {reviewer_name[0]}')
                print("\n")

            except AttributeError as e:
                print(f"Error parsing reviewer_name from {review}: {e}")
                continue  # Skip this job if data parsing fails
        
        
        if review:
            try:
                # 4, Find the relevant div that contains the review's title
                review_title = review.find('h2', class_ = 'typography_heading-xs__osRhC typography_appearance-default__t8iAq').text
                print(f'The review_title is: {review_title}')
                print("\n")


            except AttributeError as e:
                print(f"Error parsing review_title from {review}: {e}")
                continue  # Skip this job if data parsing fails
        

        if review:
            try:
                # 5, Find the relevant div that contains the review text
                review_text = review.find('p', class_ = 'typography_body-l__v5JLj typography_appearance-default__t8iAq').text
                print(f'The review_text is: {review_text}')
                print("\n")


            except AttributeError as e:
                print(f"Error parsing review_text from {review}: {e}")
                continue  # Skip this job if data parsing fails


        if review:
            try:
                # 6, Find the relevant div that contains the review Date of experience
                Date_of_experience = review.find('p', class_ = 'typography_body-m__k2UI7 typography_appearance-default__t8iAq').text
                Date_of_experience = Date_of_experience.split(':')
                Date_of_experience1 = Date_of_experience[1]
                print(f'{Date_of_experience[0]}: {Date_of_experience[1]}')
                print("\n")


            except AttributeError as e:
                print(f"Error parsing Date_of_experience from {review}: {e}")
                continue  # Skip this job if data parsing fails


        if review:
            try:
                # 7, Find the relevant div that contains the rating
                rating = review.find('div', class_ = 'star-rating_starRating__sdbkn star-rating_medium__Oj7C9')
                # print(f'The rating is: {rating}')
                # print("\n")

                if rating:
                    try:
                        # Find the <img> tag inside the div
                        img_tag = rating.find('img')
                        # print(f'The img tag is: {img_tag}')
                        # print("\n")
                        
                        if img_tag:
                            try:
                                # Extract the value of the 'alt' attribute, which contains the rating text
                                rating_text = img_tag.get('alt')
                                print(f'The rating_text is: {rating_text}')
                                print("\n")

                            except AttributeError as e:
                                print(f"Error parsing rating_text from {review}: {e}")
                                continue  # Skip this job if data parsing fails

                    
                    except AttributeError as e:
                        print(f"Error parsing img_tag from {review}: {e}")
                        continue  # Skip this job if data parsing fails

            
            except AttributeError as e:
                print(f"Error parsing rating from {review}: {e}")
                continue  # Skip this job if data parsing fails






        reviews_dict['reviewer_name'].append(reviewer_name1)
        reviews_dict['country'].append(country1)
        reviews_dict['review_title'].append(review_title)
        reviews_dict['review_text'].append(review_text)
        reviews_dict['Date_of_experience'].append(Date_of_experience1)
        reviews_dict['rating_text'].append(rating_text)
    reviews_dict['company_name'].append(company_name1)

    #This is to balance the count of company name in the dictionary, if not we'll not be able to create the dataframe table...
    # ...because it'll say table can't be created unless all dictionary values are the same quantity
    count_reviewer_name = len(reviews_dict['reviewer_name'])
    count_company_name = len(reviews_dict['company_name'])

    print(f'count_reviewer_name is: {count_reviewer_name}')
    print('\n')
    print(f'count_company_name is: {count_company_name}')


    for _ in range(count_reviewer_name - count_company_name):
        reviews_dict['company_name'].append(company_name1)


<Response [200]>


The company_name is: Grubhub


The country is: US


The reviewer_name is: Norman Martin


The review_title is: Too many delivery issues lately


The review_text is: Too many delivery issues lately. Cold food and wrong orders. Support is polite but slow to respond.


Date of experience:  June 05, 2024


The rating_text is: Rated 2 out of 5 stars


The country is: US


The reviewer_name is: Josephine Hernandez


The review_title is: Very good service!❤️


Error parsing review_text from <div class="styles_cardWrapper__kOLEb styles_show__qAseP"><article class="paper_paper__EGeEb paper_square__owXbO card_card__yyGgu card_noPadding__OOiac card_square___AZeg styles_reviewCard__rvE5E" data-service-review-card-paper="true"><div><div class="styles_reviewCardInnerHeader__dKkyc"><aside aria-label="Info for Josephine Hernandez" class="styles_consumerInfoWrapper__uQZDN"><div class="styles_consumerDetailsWrapper__LSBJS"><div class="avatar_imageWrapper__9hWrp" style="width:44px;heig

In [153]:
reviews_dict

{'company_name': ['Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'Grubhub',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'ReStockIt.com',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  'EveryPlate',
  "Burgers'",
  "Burgers'",
  "Burgers'",
  "Burgers'",
  "Burgers'",
  "Burgers'",
  "B

In [154]:
# Checking to confirm the number of Date_of_experience in the dictionary
reviews_dict['Date_of_experience']

len(reviews_dict['Date_of_experience'])

355

In [155]:
# Checking to confirm the number of company_name in the dictionary
reviews_dict['company_name']

len(reviews_dict['company_name'])

355

In [156]:
# Creating the dataframe from the dictionary
reviews_df = pd.DataFrame(reviews_dict)

reviews_df

,company_name,country,reviewer_name,review_title,review_text,Date_of_experience,rating_text
0,Grubhub,US,Norman Martin,Too many delivery issues lately,Too many delivery issues lately. Cold food and...,"June 05, 2024",Rated 2 out of 5 stars
1,Grubhub,US,Sara,I've had an order that was missing…,I've had an order that was missing items. The ...,"April 05, 2025",Rated 1 out of 5 stars
2,Grubhub,US,Jason,FAKE FOOD OFFERS,I had ordered from one restaurant a few times ...,"April 05, 2025",Rated 1 out of 5 stars
3,Grubhub,US,Andrew Neymarc,False delivery information and refusal…,False delivery information and refusal to refu...,"April 05, 2025",Rated 1 out of 5 stars
4,Grubhub,US,Consumer,By far,"By far, the worst food delivery service.","April 04, 2025",Rated 1 out of 5 stars
...,...,...,...,...,...,...,...
350,Fulton,US,Martha Quiroga,I sautéed the salmon the other night,I sautéed the salmon the other night. My husba...,"March 31, 2025",Rated 5 out of 5 stars
351,Fulton,US,Victor,Great fish even though it is a little…,Great fish even though it is a little expensiv...,"March 17, 2025",Rated 5 out of 5 stars
352,Fulton,US,Rick,All items arrived frozen.,All items arrived frozen.,"March 27, 2025",Rated 5 out of 5 stars
353,Fulton,US,Stanley,Size o trout shown and delivered not…,Size o trout shown and delivered not same and ...,"March 27, 2025",Rated 3 out of 5 stars


bs4.element.ResultSet

In [ ]:
# # Find the relevant div that contains the rating
# review = Soup.find('div', class_ = 'styles_cardWrapper__kOLEb styles_show__qAseP')

# if review:
#     rating = review.find('div', class_ = 'star-rating_starRating__sdbkn star-rating_medium__Oj7C9')
#     print(f'The rating is: {rating}')
#     print("\n")

#     if rating:
#         # Find the <img> tag inside the div
#         img_tag = rating.find('img')
#         print(f'The img tag is: {img_tag}')
#         print("\n")

#         if img_tag:
#             # Extract the value of the 'alt' attribute, which contains the rating text
#             rating_text = img_tag.get('alt')
#             print(f'The rating_text is: {rating_text}')


#             print("\n")





The rating is: <div class="star-rating_starRating__sdbkn star-rating_medium__Oj7C9"><img alt="Rated 2 out of 5 stars" src="https://cdn.trustpilot.net/brand-assets/4.1.0/stars/stars-2.svg"/></div>


The img tag is: <img alt="Rated 2 out of 5 stars" src="https://cdn.trustpilot.net/brand-assets/4.1.0/stars/stars-2.svg"/>


The rating_text is: Rated 2 out of 5 stars




In [ ]:
# # Find the relevant div that contains the reviewer's name
# review = Soup.find('div', class_ = 'styles_cardWrapper__kOLEb styles_show__qAseP')

# if review:
#     reviewer_name = review.find('a', href=re.compile(r'^/users/')).text
#     reviewer_name = reviewer_name.split(country1)
#     print(reviewer_name[0])
#     print(reviewer_name)

Juan Hernandez
['Juan Hernandez', '•4 reviews']


In [ ]:
# # Find the relevant div that contains the reviewer's country
# review = Soup.find('div', class_ = 'styles_cardWrapper__kOLEb styles_show__qAseP')

# if review:
#     country = review.find('div', class_ = 'styles_consumerExtraDetails__TylYM').text
#     country = country.split('•')
#     country1 = country[0]
#     print(country[0])

US


In [ ]:
# # Find the relevant div that contains the review's title
# review = Soup.find('div', class_ = 'styles_cardWrapper__kOLEb styles_show__qAseP')

# if review:
#     review_title = review.find('h2', class_ = 'typography_heading-xs__osRhC typography_appearance-default__t8iAq').text
#     print(review_title)



Too many delivery issues lately


In [ ]:
# # Find the relevant div that contains the review text
# review = Soup.find('div', class_ = 'styles_cardWrapper__kOLEb styles_show__qAseP')

# if review:
#     review_text = review.find('p', class_ = 'typography_body-l__v5JLj typography_appearance-default__t8iAq').text
#     print(review_text)


Too many delivery issues lately. Cold food and wrong orders. Support is polite but slow to respond.


In [ ]:
# # Find the relevant div that contains the review Date of experience
# review = Soup.find('div', class_ = 'styles_cardWrapper__kOLEb styles_show__qAseP')

# if review:
#     Date_of_exp = review.find('p', class_ = 'typography_body-m__k2UI7 typography_appearance-default__t8iAq').text
#     Date_of_exp = Date_of_exp.split(':')
#     print(f'{Date_of_exp[0]}: {Date_of_exp[1]}')


Date of experience:  June 05, 2024


In [ ]:
# # Finding the company's name
# company_name = Soup.find('h1', class_ = 'typography_default__PeVD_ typography_appearance-default__t8iAq title_title__dK8Bt').text
# company_name = company_name.split()
# print(company_name[0])

Grubhub
